In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from xgboost import XGBClassifier  # Import XGBoost

def load_data_by_batch(csv_path, batch_id):
    df = pd.read_csv(csv_path)
    df["Bộ"] = df["Bộ"].astype(int)
    batch_df = df[df["Bộ"] == batch_id]
    
    n_fft = batch_df["n_fft"].values[0]
    hop_length = batch_df["hop_length"].values[0]
    
    train_row = batch_df[batch_df["Tập"] == "train"]
    test_row = batch_df[batch_df["Tập"] == "test"]
    
    X_train = np.load(train_row["X_file_path"].values[0])
    y_train = np.load(train_row["y_file_path"].values[0])
    X_test = np.load(test_row["X_file_path"].values[0])
    y_test = np.load(test_row["y_file_path"].values[0])
    
    return X_train, y_train, X_test, y_test, n_fft, hop_length

def run_xgb_experiment(X_train, y_train, X_test, y_test, n_fft, hop_length, mfcc_steps):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1))
    X_test_scaled = scaler.transform(X_test.reshape(X_test.shape[0], -1))
    
    xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
    xgb_params = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, 10],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
    }
    
    results = []
    for i in mfcc_steps:
        X_train_subset = X_train_scaled[:, :i]
        X_test_subset = X_test_scaled[:, :i]
        
        start_time = time.time()
        random_search = RandomizedSearchCV(xgb, xgb_params, n_iter=10, cv=3, random_state=42, n_jobs=-1, verbose=1)
        random_search.fit(X_train_subset, y_train)
        
        best_params = random_search.best_params_
        y_pred = random_search.predict(X_test_subset)
        test_accuracy = accuracy_score(y_test, y_pred)
        macro_f1 = f1_score(y_test, y_pred, average='macro')
        confusion = confusion_matrix(y_test, y_pred)
        
        print(f"MFCC: {i}")
        print(confusion)
        
        elapsed_time = time.time() - start_time
        results.append({
            'numofMFCC': i,
            'n_fft': n_fft,
            'Hop_length': hop_length,
            'Best_hyperparameters': best_params,
            'test_accuracy': test_accuracy,
            'macro_f1': macro_f1,
            'time': elapsed_time
        })
    
    return pd.DataFrame(results)

# Chạy thử nghiệm với XGBoost
mfcc_steps = [13, 20, 40, 80]
csv_path = r"C:\Users\manhm\Desktop\BeeSoundClassifier\data\extracted_features\mfcc\mfcc_extraction_log.csv"

for i in range(0, 4):
    batch_id = i
    data = load_data_by_batch(csv_path, batch_id)
    
    if data is None:
        continue
    
    print(f"\n📂 Batch {batch_id}")
    print("Training XGBoost")
    X_train, y_train, X_test, y_test, n_fft, hop_length = data
    df = run_xgb_experiment(X_train, y_train, X_test, y_test, n_fft, hop_length, mfcc_steps)
    
    print(f"\n📌 Kết quả Batch {batch_id}:")
    print(df)
    
    df.to_csv(f"xgb_results_batch_{batch_id}.csv", index=False)



📂 Batch 0
Training XGBoost
Fitting 3 folds for each of 10 candidates, totalling 30 fits


c:\Users\manhm\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:22:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


MFCC: 13
[[1013   58   24]
 [ 172  291  229]
 [   5   30  938]]
Fitting 3 folds for each of 10 candidates, totalling 30 fits


c:\Users\manhm\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:22:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


MFCC: 20
[[1025   60   10]
 [ 186  290  216]
 [   4   18  951]]
Fitting 3 folds for each of 10 candidates, totalling 30 fits


...

MFCC: 20
[[1022   61   12]
 [ 189  279  224]
 [   5   18  950]]
Fitting 3 folds for each of 10 candidates, totalling 30 fits


c:\Users\manhm\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:30:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


MFCC: 40
[[1033   46   16]
 [ 197  270  225]
 [  10   19  944]]
Fitting 3 folds for each of 10 candidates, totalling 30 fits


c:\Users\manhm\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:31:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


MFCC: 80
[[1024   64    7]
 [ 174  287  231]
 [   6   27  940]]

📌 Kết quả Batch 3:
   numofMFCC  n_fft  Hop_length  \
0         13   2048        1024   
1         20   2048        1024   
2         40   2048        1024   
3         80   2048        1024   

                                Best_hyperparameters  test_accuracy  macro_f1  \
0  {'subsample': 0.7, 'n_estimators': 100, 'max_d...        0.80942  0.760807   
1  {'subsample': 0.7, 'n_estimators': 100, 'max_d...        0.81558  0.765310   
2  {'subsample': 0.7, 'n_estimators': 100, 'max_d...        0.81413  0.761828   
3  {'subsample': 0.7, 'n_estimators': 100, 'max_d...        0.81558  0.767094   

        time  
0   9.598730  
1  13.692915  
2  27.394476  
3  56.807911  
